In [ ]:
# Standard library imports
import sys, os
sys.path.append(os.path.abspath('../tools'))
# Third party imports
import pandas as pd
# Local imports
import preprocess as pre

## Read and process data ##
df = pd.read_csv('/Users/bliau/Documents/VS Code/Python/RuTransloc/Data/4017105.csv')
df = pre.convertTime(df)
df = pre.calculateETA(df)
# df = pre.encodeTime(df, 5)
df.drop('Heading', axis=1, inplace=True)
df.drop('Time', axis=1, inplace=True)
print(df.isnull().sum())

In [ ]:
# Third party imports
from sklearn.preprocessing import OneHotEncoder

## Perform one hot encoding of categorical variables ##
# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder_df = pd.DataFrame(encoder.fit_transform(df[['Vehicle_id', 'Next_stop']]).toarray())
# df = df.join(encoder_df)
# df = df.dropna()

In [ ]:
## Split data into dependent and independent variables ##
X = df.iloc[:, df.columns != 'ETA'].values
y = df.iloc[:, df.columns == 'ETA'].values

In [ ]:
# Third party imports
from sklearn.model_selection import train_test_split

## Split data into training and test sets ##
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False) # Allocates last 20% of data as test set

In [ ]:
# Third party imports
from sklearn.ensemble import RandomForestRegressor

## Train RFR model on the training set ##
regressor = RandomForestRegressor(n_estimators=500, max_depth=8, min_samples_leaf=2, random_state=42)
regressor.fit(X_train, y_train)

In [ ]:
## Predicting the test set results ##
y_pred = regressor.predict(X_test)
print(y_pred)

In [ ]:
## Determine feature importance ##
from sklearn.inspection import permutation_importance
results = permutation_importance(regressor, X, y, scoring='neg_mean_squared_error')
importance = results.importances_mean
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
# Third party imports
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Calculate accuracy ##
r2 = r2_score(y_test, y_pred)
print('R-squared score:', r2)
mae = mean_absolute_error(y_test, y_pred)
print('Mean absolute error:', mae)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

In [ ]:
# Third party imports
import pickle

## Save trained model ##
pickle.dump(regressor, open('randomForest', 'wb'))

In [ ]:
## Load trained model ##
model = pickle.load(open('randomForest', 'rb'))
result = model.score(X_test, y_test)